In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def f(x):
    return 3*x**2 - 4*x + 5

In [ ]:
f(3.0)

In [ ]:
# https://numpy.org/doc/stable/reference/generated/numpy.arange.html
xs = np.arange(-5, 5, 0.25)
print("xs")
print(xs)
ys = f(xs)
print("ys")
print(ys)
plt.plot(xs, ys)

In [ ]:
h = 0.000001
x = 3.0
(f(x + h) - f(x)) / h

# The formula above 3*x**2 - 4*x + 5
# has derivative 6x - 4
# which if given x == 3.0 is 14

In [ ]:
h = 0.0001

# inputs
a = 2.0
b = -3.0
c = 10.0

d1 = a*b + c
c += h
d2 = a*b + c

print('d1', d1)
print('d2', d2)
print('slope', (d2 - d1)/h)

In [111]:
class Value:
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0
        # internal variables used for autograd graph construction
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op # the op that produced this node, for graphviz / debugging / etc
        self.label = label

    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            # self.grad += out.grad
            # other.grad += out.grad
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward

        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            # self.grad += other.data * out.grad
            # other.grad += self.data * out.grad
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward

        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data**other, (self,), f'**{other}')

        def _backward():
            self.grad += (other * self.data**(other-1)) * out.grad
        out._backward = _backward

        return out

    def __truediv__(self, other):
        return self * other**-1

    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        out = Value(t, (self, ), 'tanh')

        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward

        return out

    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self, ), 'exp')

        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward

        return out

    def relu(self):
        out = Value(0 if self.data < 0 else self.data, (self,), 'ReLU')

        def _backward():
            self.grad += (out.data > 0) * out.grad
        out._backward = _backward

        return out

    def __neg__(self): # -self
        return self * -1

    def __radd__(self, other): # other + self
        return self + other

    def __sub__(self, other): # self - other
        return self + (-other)

    def __rsub__(self, other): # other - self
        return other + (-self)

    def __rmul__(self, other): # other * self
        return self * other

    def __truediv__(self, other): # self / other
        return self * other**-1

    def __rtruediv__(self, other): # other / self
        return other * self**-1

    def backward(self, debug=False):
        topo = []
        visited = set()
        def build_topo(v: Value):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        # go one variable at a time and apply the chain rule to get its gradient
        self.grad = 1
        for v in reversed(topo):
            if debug:
                print(f"DEBUG: _backward() for {v.label} / {v.data}")
            v._backward()

In [ ]:
a = Value(2.0, label='a')
b = Value(-3.0, label='b')
c = Value(10.0, label='c')
e = a*b; e.label = 'e'
d = e + c; d.label = 'd'
f = Value(-2.0, label='f')
L = d * f; L.label = 'L'
print(d)

In [ ]:
from graphviz import Digraph

def trace(root):
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root, format='svg', rankdir='LR'):
    """
    format: png | svg | ...
    rankdir: TB (top to bottom graph) | LR (left to right)
    """
    assert rankdir in ['LR', 'TB']
    nodes, edges = trace(root)
    dot = Digraph(format=format, graph_attr={'rankdir': rankdir}) #, node_attr={'rankdir': 'TB'})

    for n in nodes:
        dot.node(name=str(id(n)), label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
        if n._op:
            dot.node(name=str(id(n)) + n._op, label=n._op)
            dot.edge(str(id(n)) + n._op, str(id(n)))

    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)

    return dot

In [ ]:
# Need to run:
# brew install graphviz

draw_dot(L)

In [ ]:
test01_x = np.arange(-5, 5, 0.2)
plt.plot(test01_x, np.tanh(test01_x))
plt.grid()

In [ ]:
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')

w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')

b = Value(6.8813735870195432, label='b')

x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1w1+x2w2'

n = x1w1x2w2 + b; n.label = 'n'

e = (2*n).exp(); e.label = 'e'
o = (e - 1) / (e + 1)
o.label = 'o'

In [ ]:
# The gradient for the output is always since the derivative with respect to itself is 1
# o.grad = 1.0
# o._backward()
# n._backward()
# b._backward()
# x1w1x2w2._backward()
# x1w1._backward()
# x2w2._backward()
# x1._backward()
# w1._backward()
# x2._backward()
# w2._backward()

# o.backward() is essentially the topo sorted list from the manual back-propagation above
o.backward(debug=True)

In [ ]:
draw_dot(o)

In [ ]:
# pytorch comparison to the above code
import torch

x1 = torch.Tensor([2.0]).double()
x1.requires_grad = True
x2 = torch.Tensor([0.0]).double()
x2.requires_grad = True

w1 = torch.Tensor([-3.0]).double()
w1.requires_grad = True
w2 = torch.Tensor([1.0]).double()
w2.requires_grad = True

b = torch.Tensor([6.8813735870195432]).double()
b.requires_grad = True

n = x1*w1 + x2*w2 + b
o = torch.tanh(n)

print(o.data.item())

o.backward()

print("---")
print('x2', x2.grad.item())
print('w2', w2.grad.item())
print('x1', x1.grad.item())
print('w1', w1.grad.item())

In [112]:
import random

class Module:
    def zero_grad(self):
        for p in self.parameters():
            p.grad = 0

    def parameters(self):
        return []

class Neuron(Module):
    def __init__(self, nin, nonlin=True):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(0)
        self.nonlin = nonlin

    def __call__(self, x):
        act = sum((wi*xi for wi,xi in zip(self.w, x)), self.b)
        return act.relu() if self.nonlin else act

    def parameters(self):
        return self.w + [self.b]

    def __repr__(self):
        return f"{'ReLU' if self.nonlin else 'Linear'}Neuron({len(self.w)})"

class Layer(Module):
    def __init__(self, nin, nout, **kwargs):
        self.neurons = [Neuron(nin, **kwargs) for _ in range(nout)]

    def __call__(self, x):
        out = [n(x) for n in self.neurons]
        return out[0] if len(out) == 1 else out

    def parameters(self):
        return [p for n in self.neurons for p in n.parameters()]

    def __repr__(self):
        return f"Layer of [{', '.join(str(n) for n in self.neurons)}]"

class MultilayerPerceptron(Module):
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1], nonlin=i!=len(nouts)-1) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

    def __repr__(self):
        return f"MLP of [{', '.join(str(layer) for layer in self.layers)}]"

In [ ]:
# x = [2.0, 3.0]
# n = Layer(2, 3)
# n(x)

tx = [2.0, 3.0, -1.0]
tn = MultilayerPerceptron(3, [4, 4, 1])
to = tn(tx)

print(to)

In [ ]:
draw_dot(to)

In [ ]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0]
ypred = [tn(x) for x in xs]
print(ypred)

In [105]:
# [print(ygt, yout) for ygt, yout in zip(ys, ypred)]

# [(yout - ygt)**2 for ygt, yout in zip(ys, ypred)]

loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))
print(loss)

loss.backward()

# draw_dot(loss)

print(tn.layers[0].neurons[0].w[0].grad)

Value(data=1.3952109443570644, grad=0.0)
86.32067239321333


In [138]:
epochs = 100
learning_rate = 0.001

model = MultilayerPerceptron(3, [4, 4, 1])

for epoch in range(epochs):
    xs = [
        [2.0, 3.0, -1.0],
        [3.0, -1.0, 0.5],
        [0.5, 1.0, 1.0],
        [1.0, 1.0, -1.0],
    ]
    ys = [1.0, -1.0, -1.0, 1.0]

    ## Forward Pass
    ypred = [model(x) for x in xs]

    if epoch % 10 == 0:
        print("--- predictions ---")
        print(ypred)

    loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))

    if epoch % 10 == 0:
        print("loss:", epoch, loss.data)

    ## Backward Pass
    for p in model.parameters():
        p.grad = 0.0
    loss.backward()

    ## Update
    for p in model.parameters():
        p.data += (-learning_rate * p.grad)

--- predictions ---
[Value(data=-1.202149972898547, grad=0.0), Value(data=-0.2092544464030927, grad=0.0), Value(data=-0.29943680349392904, grad=0.0), Value(data=-0.5282698182867913, grad=0.0)]
loss: 0 8.301140463455496
--- predictions ---
[Value(data=0.6380020768397694, grad=0.0), Value(data=-0.05351750796783561, grad=0.0), Value(data=-0.07760134292848475, grad=0.0), Value(data=0.3431572062734387, grad=0.0)]
loss: 10 2.309133342333585
--- predictions ---
[Value(data=0.9492238710719652, grad=0.0), Value(data=-0.09471445297431946, grad=0.0), Value(data=-0.10982089764479896, grad=0.0), Value(data=0.4631577642135418, grad=0.0)]
loss: 20 1.9027385573166167
--- predictions ---
[Value(data=1.0177127152165393, grad=0.0), Value(data=-0.16147662004954738, grad=0.0), Value(data=-0.17459347961544952, grad=0.0), Value(data=0.4673290752262782, grad=0.0)]
loss: 30 1.6684694369964965
--- predictions ---
[Value(data=1.0490166809385015, grad=0.0), Value(data=-0.2275329616380794, grad=0.0), Value(data=-0

Value(data=5.168250102026892, grad=0.0)
